In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q simpletransformers

In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
import os
import tarfile
import numpy as np

In [ ]:
def unpack_model(model_name=''):
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('drive/MyDrive/bert_model')

In [ ]:
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 15}

model = ClassificationModel(
    "bert", "/bert_model/",
    num_labels=39,
    args=train_args,
    use_cuda=False
)

In [ ]:
def flatten(file):
  return [id for ids in file for id in ids] 

df = pd.read_csv('test.csv', sep='|')

file_id_list = []
id_list = []
class_list = []

for i in range(10):
  row = df.iloc[i, :].dropna()

  if row[-1][:2] != '10':
    file_id = [int(row[-1][0])] * row[:-1].shape[0]
  
  else: file_id = [int(row[-1][:2])] * row[:-1].shape[0]
  
  id = list(row[:-1].index)
  predictions, raw_outputs = model.predict(list(row[:-1].values))
  result = list(map(lambda x: x+1, predictions))
  file_id_list.append(file_id)
  id_list.append(id)
  class_list.append(result)

file_id = flatten(file_id_list)
id = flatten(id_list)
results = flatten(class_list)

In [ ]:
data = {'file_id': file_id, 'id': id, 'class': results}
fin_df = pd.DataFrame(data=data).astype('int').sort_values(by=['file_id', 'id'])
fin_df.to_csv('Liberal_Python.csv', index=False, sep=';')